In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
# user defined parameters
pool = 2 # The pool the armor to be augmented belongs to
init_budget = 18 # The initial budget of the armor piece

rng_sd = 0 # Seed for rng
rng = np.random.default_rng(rng_sd)

In [3]:
data = []
with open(f'./data/aug_val_pool_{pool}.csv', 'r') as f:
    header = f.readline()
    line = f.readline()
    while line:
        entries = line.split(',')
        data.append((entries[0], int(entries[1]), int(entries[2]), int(entries[3])))
        line = f.readline()

In [4]:
def get_qurio_aug(init_budget):
    def allowed_upgrade(roll, upgrade):
        if roll == 0:
            # first roll must be defense
            if upgrade[:-1] == 'Defense':
                return True
            else:
                return False
        elif roll == 1:
            # second roll must be skill
            if upgrade[:-1] == 'Skill':
                return True
            else:
                return False
        else:
            return True

    def clean_up_def():
        # drop all defense augments if they sum up to 0
        # return True if augs have been cleaned up, else return False
        def_ind = []
        total_def = 0
        for ind, (upgrade, lv, cost) in enumerate(total_augs):
            if upgrade[:-1] == 'Defense':
                def_ind.append(ind)
                total_def += lv
        if total_def == 0 and len(def_ind) > 0:
            for ind in def_ind[::-1]:
                del total_augs[ind]
            return True
        else:
            return False

    def remaining_budget():
        return init_budget - sum([cost for upgrade, lv, cost in total_augs])

    total_augs = []
    for roll in range(50):
        if len(total_augs) == 6:
            for upgrade, lv, prob, cost in data:
                if upgrade[:-1] == 'Defense' and cost == remaining_budget():
                    total_augs.append((upgrade, lv, cost))
            if clean_up_def():
                continue
            else:
                break

        allowed_augs = []
        for upgrade, lv, prob, cost in data:
            if allowed_upgrade(roll, upgrade):
                allowed_augs.extend([(upgrade, lv, cost)] * prob)
        chosen_ind = rng.integers(0, high=len(allowed_augs))
        chosen_upgrade, chosen_lv, chosen_cost = allowed_augs[chosen_ind]
        if chosen_cost > remaining_budget():
            continue
        total_augs.append((chosen_upgrade, chosen_lv, chosen_cost))
        if clean_up_def():
            continue

        if remaining_budget() == 0:
            break

    return total_augs

In [5]:
get_qurio_aug(init_budget)

[('Defense-', -6, -3),
 ('Skill+', 1, 9),
 ('Water res-', -3, -3),
 ('Defense+', 22, 15)]